### Installa le dipendenze

In [ ]:
%pip install --quiet sonar-space seaborn pandas

Caricamento dei modelli Sonar

In [ ]:
import torch

from sonar.inference_pipelines.text import (
    TextToEmbeddingModelPipeline,
    TextToTextModelPipeline,
)

Inizializzazione dei modelli

In [ ]:
text2vec = TextToEmbeddingModelPipeline(
    encoder="text_sonar_basic_encoder", tokenizer="text_sonar_basic_encoder"
)
text2text = TextToTextModelPipeline(
    encoder="text_sonar_basic_encoder",
    decoder="text_sonar_basic_decoder",
    tokenizer="text_sonar_basic_encoder",
)  # tokenizer is attached to both encoder and decoder cards

 Definizione di frasi di esempio

In [ ]:
italian_verses = [
    "Nel mezzo del cammin di nostra vita",
    "Mi ritrovai per una selva oscura",
    "Che la diritta via era smarrita.",
    "Ahi quanto a dir qual era è cosa dura",
    "Esta selva selvaggia e aspra e forte",
    "Che nel pensier rinova la paura!",
    "Tant'era amara che poco è più morte",
    "Ma per trattar del ben ch'i' vi trovai",
    "Dirò de l'altre cose ch'i' v'ho scorte."
]

# Traduzione inglese (John Ciardi, adattata)
english_verses = [
    "Midway upon the journey of our life",
    "I found myself within a forest dark",
    "For the straightforward pathway had been lost.",
    "Ah me! how hard a thing it is to say",
    "What was this forest savage, rough, and stern",
    "Which in the very thought renews the fear.",
    "So bitter is it, death is little more.",
    "But of the good to treat which there I found",
    "Speak will I of the other things I saw there."
]

Ottenere embeddings per le frasi

In [ ]:
output_ita = text2vec.predict(italian_verses, source_lang="ita_Latn")
embed_italian = torch.nn.functional.normalize(output_ita, dim=-1).numpy()

output_eng = text2vec.predict(english_verses, source_lang="eng_Latn")
embed_english = torch.nn.functional.normalize(output_eng, dim=-1).numpy()

Calcolo della similarità tra frasi

In [ ]:
#!pip install seaborn pandas
import pandas as pd
import seaborn as sns

# Calcolo similarità coseno
cos_sim = embed_italian.dot(embed_english.T)
sim_df = pd.DataFrame(cos_sim, columns=english_verses, index=italian_verses)

plt.figure(figsize=(10, 8))
sns.heatmap(sim_df, cmap="vlag", annot=True, fmt=".2f", linewidths=0.5)
plt.title("Cosine Similarity: Divina Commedia (Italiano ↔ Inglese)")
plt.xlabel("Versi in Inglese")
plt.ylabel("Versi in Italiano")
plt.show()

Traduzione delle frasi con Sonar

In [ ]:
inglese_tradotto = text2text.predict(italian_verses, source_lang="ita_Latn", target_lang="eng_Latn")
italiano_tradotto = text2text.predict(english_verses, source_lang="eng_Latn", target_lang="ita_Latn")

Valutazione della qualità della traduzione con BLEU

In [ ]:
import sacrebleu

# Calcolo BLEU score
bleu_ita_to_eng = sacrebleu.corpus_bleu(inglese_tradotto, [english_verses], tokenize="flores200")
bleu_eng_to_ita = sacrebleu.corpus_bleu(italiano_tradotto, [italian_verses], tokenize="flores200")

# Output BLEU scores
print("*" * 100)
print("Italiano → Inglese BLEU score:")
print(bleu_ita_to_eng)
print("*" * 100)
print("Inglese → Italiano BLEU score:")
print(bleu_eng_to_ita)